In [1]:
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm.auto import tqdm
import datasets
import neptune
from loss_new import SGTLoss

import json

import hydra
from omegaconf import DictConfig, OmegaConf

from metrics import cos_metric, topk_intersection_metric, reconstruction_rank_metric

from collections import defaultdict

from dataset import SGTDataset

from sgt_model import SGTModel

/home/alex/research/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda:0')

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

llm = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", device_map=device)

In [4]:
ds = load_dataset("ai2_arc", "ARC-Challenge")['train']

In [10]:
llm.model.embed_tokens.weight.shape

torch.Size([151936, 1536])

In [11]:
llm.model.embed_tokens

Embedding(151936, 1536)